In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

In [2]:
import torch
from utils.tokenizer import CharTokenizer
from model.gpt import GPT, GPTConfig

# Szöveg beolvasása
with open("../data/input.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Tokenizer + encode
tokenizer = CharTokenizer(text)
sample = "Let them"
tokens = tokenizer.encode(sample)
x = torch.tensor(tokens).unsqueeze(0)  # shape: (1, T)

# Modell inicializálás
config = GPTConfig(vocab_size=tokenizer.vocab_size, block_size=128)
model = GPT(config)

# Forward pass
with torch.no_grad():
    logits = model(x)  # shape: (1, T, vocab_size)
    last_logits = logits[0, -1]  # csak az utolsó pozíció logitjai

# Softmax valószínűség
probs = torch.softmax(last_logits, dim=-1)

# Legvalószínűbb token
predicted_token_id = torch.argmax(probs).item()
predicted_char = tokenizer.itos[predicted_token_id]

print("Bemenet:", sample)
print("Tokenek:", tokens)
print("Következő karakter predikció (logit alapján):", predicted_char)
print("Top 5 valószínűség:")
top5 = torch.topk(probs, 5)
for idx, prob in zip(top5.indices, top5.values):
    print(f"{tokenizer.itos[idx.item()]} → {prob.item():.4f}")


Bemenet: Let them
Tokenek: [45, 68, 83, 2, 83, 71, 68, 76]
Következő karakter predikció (logit alapján): ”
Top 5 valószínűség:
” → 0.0262
Z → 0.0223
w → 0.0210
O → 0.0194
\ → 0.0188
